# k Nearest Neighbors Example and Extensions

In [2]:
%matplotlib inline 

from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
from scipy.spatial import distance
from operator import itemgetter
from collections import defaultdict

import numpy as np
import matplotlib.pyplot as plt


In [3]:
# Ronald Fisher's Iris Data Set 
iris = datasets.load_iris()
iris_X = iris.data
iris_Y = iris.target


In [4]:
# We split the iris dataset into training data and test data.
np.random.seed(1000)
indices = np.random.permutation(len(iris_X))
iris_data_train = iris_X[indices[:-20]]
iris_target_train = iris_Y[indices[:-20]]
iris_data_test  = iris_X[indices[-20:]]
iris_target_test  = iris_Y[indices[-20:]]

## SKLearn's Built in k-NN Classifier

In [5]:
# Basic configuration of the sklearn k-NN classifier.
knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', p=2, metric='minkowski')
knn.fit(iris_data_train, iris_target_train)

print(knn.predict(iris_data_test))
print(iris_target_test)

[1 1 0 2 1 0 1 0 0 2 2 0 0 1 2 0 1 1 1 1]
[1 1 0 2 1 0 1 0 0 2 2 0 0 1 2 0 1 1 1 1]


# Basic Implementation

In [6]:
class knearestbasic():
    """
    Basic implementation of the k-Nearest Neighbor classification algorithm
    as described in https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm
    """
    
    def __init__(self, training_data, training_targets):
        """
        Initializes the classifier with the given training data and
        classes.
        """
        self.training_data = training_data
        self.training_targets = training_targets
        
        self._classes = set(training_targets)
        self._num_classes = len(self._classes)
        
        self._features = len(training_data[0])
        
    def classify(self, cases, k=5, dist_func=distance.euclidean):
        """
        Classify a set of cases using training data.
        """
        results = []
        for case in cases:
            neighbors = self.__get_neighbors(case, k, dist_func)
            vote = self.__count_votes(neighbors)[0][0]
            results.append(vote)
        return results
    
    def __get_neighbors(self, case, k, dist_func):
        """
        Classifies a single case using training data.
        """
        if (len(case) != self._features):
            raise ValueError("invalid case")
            
        distances = []
        for i in range(len(self.training_data)):
            dist = dist_func(case, self.training_data[i])
            distances.append([self.training_targets[i], dist])

        distances.sort(key=itemgetter(1))
        return distances[0:k]
    
    def __count_votes(self, neighbors):
        """
        Counts votes for classes and handles ties.
        """
        votes = {}
        for i in range(len(neighbors)):
            cl = neighbors[i][0]
            if cl in votes:
                votes[cl] += 1
            else:
                votes[cl] = 1
        sorted_votes = sorted(votes.items(), key=itemgetter(1), reverse=True)

        # Break ties by counting fewer neighbors
        if len(sorted_votes) > 1 and sorted_votes[0][1] == sorted_votes[1][1]:
            return self.__count_votes(neighbors[:-1])

        return sorted_votes
        
        

In [7]:
knn = knearestbasic(iris_data_train, iris_target_train)
print(str(knn.classify(iris_data_test, 10)).replace(',',''))
print(iris_target_test)

[1 1 0 2 1 0 1 0 0 2 2 0 0 1 2 0 1 1 1 1]
[1 1 0 2 1 0 1 0 0 2 2 0 0 1 2 0 1 1 1 1]


## Measuring Quality
Since we are interested in the quality as well as the accuracy of our classifying methods, we include mechanisms for calculating the Degree of Certainty and Net reliability of our classifiers - demonstrated using the Variable K Nearest Neightbor Classifier. 

In [8]:
def degree_of_certainty(scores):
    """
    Calculates the degree of certainty given a set of classification
    scores. In our model, these are the results returned by the
    __get_neighbors function. e.g. List of [class, classification_score]
    """
    max_score = 0
    total = 0
    for i in range(len(scores)):
        if (scores[i][1] > max_score):
            max_score = scores[i][1]
        total += scores[i][1]
        
    return max_score/total
    
    
test_scores = [(2, 8), (1, 2), (0, 1)]
print(degree_of_certainty(test_scores))

0.7272727272727273


## Variable k Nearest

This classifier uses the degree of certainty to compute the ideal value of k for a given training set.

In [9]:
class variableknn():
    """
    Implementation of the variable KNN detection algorithm. Which precomputes
    the ideal k value for your training set.
    """
    
    def __init__(self, training_data, training_targets, dist_func=distance.euclidean):
        """
        Initializes the classifier with the given training data and
        classes.
        """
        self.training_data = training_data
        self.training_targets = training_targets
        self.dist_func = dist_func
        
        self._classes = set(training_targets)
        self._num_classes = len(self._classes)
        
        self._features = len(training_data[0])
        
        # Precalculate optimum k values
        self.__calc_k_values()
   

    def classify(self, cases):
        results = []
        for case in cases:
            neighbors = self.__get_neighbors(case, self._k, False)
            vote = self.__class_scores(neighbors)[0][0]
            results.append(vote)
        return results
    
            
    def __calc_k_values(self):
        """
        For each of the training set elements a classification of it is performed based on a
        certain k value. We then find the average DC for each of those points and maximize that
        w.r.t. a changing k.
        
        To keep this simple we're just checking each k value for 1 < k < 30.
        """
        self._k = {cl : 0 for cl in set(self.training_targets)}
        for i in range(5, 30):
            self.__check_k(i)
        #print(self._k)
        
        
    def __check_k(self, k):
        """
        This function computes the average degree of certainty for each element in the set
        given a certain k.
        """
        dc_scores = {}
        dc_counts = {}
        for i in range(len(self.training_data)):
            if self.training_targets[i] in dc_counts:
                dc_counts[self.training_targets[i]] += 1
            else:
                dc_counts[self.training_targets[i]] = 1
                dc_scores[self.training_targets[i]] = 0
                
            neighbors = self.__get_neighbors(self.training_data[i], k, True)
            scores = self.__class_scores(neighbors)
            dc = degree_of_certainty(scores)
            dc_scores[self.training_targets[i]] += dc
            
        for cl in dc_counts:
            dc_scores[cl] = dc_scores[cl]/dc_counts[cl]
            if self._k[cl] < dc_scores[cl]:
                self._k[cl] = k
        #print(dc_scores)
    
    
    def __get_neighbors(self, case, k, skip):
        if (len(case) != self._features):
            raise ValueError("invalid case")
            
        distances = []
        
        for i in range(len(self.training_data)):
            dist = self.dist_func(case, self.training_data[i])
            if dist == 0 and skip:
                continue
            distances.append([self.training_targets[i], dist])

        distances.sort(key=itemgetter(1))
        
        if skip:
            # For precalculation
            return distances[0:k]
        else:
            # Otherwise lookup the optimal value of k for the nearest neighbor, then
            # use that value.
            opt_k = self._k[distances[0][0]]
            return distances[0:opt_k]
    
    def __class_scores(self, neighbors):
        votes = {}
        for i in range(len(neighbors)):
            cl = neighbors[i][0]
            if cl in votes:
                votes[cl] += 1
            else:
                votes[cl] = 1
        sorted_votes = sorted(votes.items(), key=itemgetter(1), reverse=True)

        # Break ties by counting fewer neighbors
        if len(sorted_votes) > 1 and sorted_votes[0][1] == sorted_votes[1][1]:
            return self.__class_scores(neighbors[:-1])
        
        return sorted_votes

In [ ]:
knn = variableknn(iris_data_train, iris_target_train)

In [ ]:
print(str(knn.classify(iris_data_test)).replace(',',''))
print(iris_target_test)

# Different Data Sets

Looking at classifying some other data sets as well: